# 02 - Merge matches and bookmaker odds

In this notebook, we merge the cleaned match statistics dataset with the cleaned bookmaker odds dataset for the 2022–2023 Premier League season.

Goals:
- Load `clean_matches_22_23.csv` and `clean_bookmakers_22_23.csv`
- Create match result based on final score
- Check that everything is good before the merger
- Prepare a merged dataset that combines match stats and betting odds for each match

In [77]:
import pandas as pd

# Display more columns when printing dataframes
pd.set_option("display.max_columns", 50)

# Load cleaned match statistics
matches_df = pd.read_csv("../data/processed/clean_matches_22_23.csv")
# Load cleaned bookmaker odds
bookmakers_df = pd.read_csv("../data/processed/clean_bookmakers_22_23.csv")

# Quick sanity check: number of rows and columns
print("Matches shape:", matches_df.shape)
print("Bookmakers shape:", bookmakers_df.shape)
print("Bookmakers columns:")
print(list(bookmakers_df.columns))

Matches shape: (380, 13)
Bookmakers shape: (380, 9)
Bookmakers columns:
['date', 'home_team', 'away_team', 'home_odds', 'draw_odds', 'away_odds', 'prob_book_home', 'prob_book_draw', 'prob_book_away']


In [78]:
# Check that both datasets now use the same set of team names (home + away) 
teams_matches = set(matches_df["home_team"]).union(matches_df["away_team"])
teams_book = set(bookmakers_df["home_team"]).union(bookmakers_df["away_team"])

print("Only in bookmakers:", sorted(teams_book - teams_matches))
print("Only in matches:", sorted(teams_matches - teams_book))

Only in bookmakers: []
Only in matches: []


In [81]:
# Check for duplicate keys in matches
dup_matches = matches_df.duplicated(subset=["date", "home_team", "away_team"]).sum()
# Check for duplicate keys in bookmakers_clean
dup_books = bookmakers_df.duplicated(subset=["date", "home_team", "away_team"]).sum()

print("Duplicate keys in matches_df:", dup_matches)
print("Duplicate keys in bookmakers_clean:", dup_books)

Duplicate keys in matches_df: 0
Duplicate keys in bookmakers_clean: 0


In [ ]:
# --- Merge match stats with bookmaker odds ---

# We merge on the three key columns that uniquely identify a match:
# - date
# - home_team
# - away_team
# "inner" keeps only matches that exist in both dataframes
# "validate='one_to_one'" ensures that there is at most one row per key in each df

merged_df = matches_df.merge(
    bookmakers_df,
    on=["date", "home_team", "away_team"],
    how="inner",
    validate="one_to_one",
    )

# Quick sanity checks
print("matches_df shape:", matches_df.shape)
print("bookmakers_clean shape:", bookmakers_df.shape)
print("merged_df shape:", merged_df.shape)

# Show the first few rows to inspect the result
merged_df.head()


In [82]:
# --- Create match result (H/D/A) and select final wide columns ---

# Create result column based on final score
# H = home win, D = draw, A = away win
def compute_result(row):
    if row["home_goals"] > row["away_goals"]:
        return "H"
    elif row["home_goals"] < row["away_goals"]:
        return "A"
    else:
        return "D"

merged_df["result"] = merged_df.apply(compute_result, axis=1)

# Keep only the columns we need for the next steps
matches_wide = merged_df[[
    "date",
    "home_team", "away_team",
    "result",
    "home_goals", "away_goals",
    "home_xg", "away_xg",
    "home_shots", "away_shots",
    "home_sot", "away_sot",
    "home_poss", "away_poss",
    "home_odds", "draw_odds", "away_odds",
    "prob_book_home", "prob_book_draw", "prob_book_away",
]]

print(matches_wide.shape)
matches_wide.head()


(380, 20)


,date,home_team,away_team,result,home_goals,away_goals,home_xg,away_xg,home_shots,away_shots,home_sot,away_sot,home_poss,away_poss,home_odds,draw_odds,away_odds,prob_book_home,prob_book_draw,prob_book_away
0,2022-08-05,Crystal Palace,Arsenal,A,0,2,1.2,1.0,10,10,2,2,56,44,4.366667,3.583333,1.856667,0.218795,0.266625,0.514581
1,2022-08-06,Tottenham Hotspur,Southampton,H,4,1,1.5,0.5,18,10,8,2,58,42,1.350000,5.296667,8.703333,0.709225,0.180765,0.110010
2,2022-08-06,Fulham,Liverpool,D,2,2,1.2,1.2,8,11,2,3,33,67,11.400000,5.906667,1.266667,0.083822,0.161779,0.754399
3,2022-08-06,Bournemouth,Aston Villa,H,2,0,0.6,0.7,7,15,3,2,35,65,3.810000,3.460000,2.030000,0.251382,0.276811,0.471806
4,2022-08-06,Newcastle United,Nottingham Forest,H,2,0,1.7,0.3,23,5,9,0,61,39,1.680000,3.746667,5.443333,0.569141,0.255202,0.175657


In [83]:
# --- Add match_id and season columns to matches_wide ---

# Make a copy to be safe
matches_wide = matches_wide.copy()

# Unique id per match: 1..n_matches
matches_wide["match_id"] = range(1, len(matches_wide) + 1)

# Season label
matches_wide["season"] = "2022-2023"

# Quick check
matches_wide[[
    "match_id",
    "season",
    "date",
    "home_team",
    "away_team",
    "result",
]].head()


,match_id,season,date,home_team,away_team,result
0,1,2022-2023,2022-08-05,Crystal Palace,Arsenal,A
1,2,2022-2023,2022-08-06,Tottenham Hotspur,Southampton,H
2,3,2022-2023,2022-08-06,Fulham,Liverpool,D
3,4,2022-2023,2022-08-06,Bournemouth,Aston Villa,H
4,5,2022-2023,2022-08-06,Newcastle United,Nottingham Forest,H


In [88]:
# --- Convert from wide format (one row per match) to long format (two rows per match) ---

rows = []

for index, row in matches_wide.iterrows():

    # Home team row
    rows.append({
        "match_id": row["match_id"],
        "season": row["season"],
        "date": row["date"],
        "team": row["home_team"],
        "opponent": row["away_team"],
        "is_home": 1,
        "goals_for": row["home_goals"],
        "goals_against": row["away_goals"],
        "xg_for": row["home_xg"],
        "xg_against": row["away_xg"],
        "shots_for": row["home_sot"],
        "shots_against": row["away_sot"],
        "poss": row["home_poss"],
        "prob_book_home": row["prob_book_home"],
        "prob_book_draw": row["prob_book_draw"],
        "prob_book_away": row["prob_book_away"],
        "result": row["result"],
    })

    # Away team row
    rows.append({
        "match_id": row["match_id"],
        "season": row["season"],
        "date": row["date"],
        "team": row["away_team"],
        "opponent": row["home_team"],
        "is_home": 0,
        "goals_for": row["away_goals"],
        "goals_against": row["home_goals"],
        "xg_for": row["away_xg"],
        "xg_against": row["home_xg"],
        "shots_for": row["away_sot"],
        "shots_against": row["home_sot"],
        "poss": row["away_poss"],
        "prob_book_home": row["prob_book_home"],
        "prob_book_draw": row["prob_book_draw"],
        "prob_book_away": row["prob_book_away"],
        "result": row["result"],
    })

# Convert to DataFrame
matches_long = pd.DataFrame(rows)

# Quick check
print(matches_long.shape)  # should be 760 (38*10*2)
matches_long.head(10) 



(760, 17)


,match_id,season,date,team,opponent,is_home,goals_for,goals_against,xg_for,xg_against,shots_for,shots_against,poss,prob_book_home,prob_book_draw,prob_book_away,result
0,1,2022-2023,2022-08-05,Crystal Palace,Arsenal,1,0,2,1.2,1.0,2,2,56,0.218795,0.266625,0.514581,A
1,1,2022-2023,2022-08-05,Arsenal,Crystal Palace,0,2,0,1.0,1.2,2,2,44,0.218795,0.266625,0.514581,A
2,2,2022-2023,2022-08-06,Tottenham Hotspur,Southampton,1,4,1,1.5,0.5,8,2,58,0.709225,0.180765,0.110010,H
3,2,2022-2023,2022-08-06,Southampton,Tottenham Hotspur,0,1,4,0.5,1.5,2,8,42,0.709225,0.180765,0.110010,H
4,3,2022-2023,2022-08-06,Fulham,Liverpool,1,2,2,1.2,1.2,2,3,33,0.083822,0.161779,0.754399,D
5,3,2022-2023,2022-08-06,Liverpool,Fulham,0,2,2,1.2,1.2,3,2,67,0.083822,0.161779,0.754399,D
6,4,2022-2023,2022-08-06,Bournemouth,Aston Villa,1,2,0,0.6,0.7,3,2,35,0.251382,0.276811,0.471806,H
7,4,2022-2023,2022-08-06,Aston Villa,Bournemouth,0,0,2,0.7,0.6,2,3,65,0.251382,0.276811,0.471806,H
8,5,2022-2023,2022-08-06,Newcastle United,Nottingham Forest,1,2,0,1.7,0.3,9,0,61,0.569141,0.255202,0.175657,H
9,5,2022-2023,2022-08-06,Nottingham Forest,Newcastle United,0,0,2,0.3,1.7,0,9,39,0.569141,0.255202,0.175657,H
